## Setup / Step 0

1. Load some basic libraries and set environment variables for AWS access
2. Initiate tileset to test

In [1]:
%%capture
!pip install -r requirements.txt
!pip install morecantile==3.4.0 loguru titiler titiler-pgstac
!pip install psycopg psycopg_binary psycopg_pool

In [24]:
import morecantile
import numpy as np
import pandas as pd
from profiler.main import Timer
import sys
sys.path.append('..')

In [33]:
tms = morecantile.tms.get("WebMercatorQuad")
zooms = range(12) # Zoom 10 is the level at which you can see large roads, 15 is buildings
xyz_tiles = []
for z in zooms:
    tile = tms.tile(57, 28, z)
    xyz_tiles.append((tile.x, tile.y, tile.z))

xyz_tiles

[(0, 0, 0),
 (1, 0, 1),
 (2, 1, 2),
 (5, 3, 3),
 (10, 6, 4),
 (21, 13, 5),
 (42, 26, 6),
 (84, 53, 7),
 (168, 107, 8),
 (337, 214, 9),
 (674, 428, 10),
 (1348, 857, 11)]

# Profile titiler-xarray

In [7]:
%load_ext autoreload
%autoreload

import xarray_tile_reader
import zarr_reader
import zarr_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
shapes = ['180x360', '569x1138', '1799x3599', '5689x11379', 'one_tenth_planet']

In [35]:
variable = 'data'
datastores = {}
for shape in shapes:
    store_dir = f"store_{shape}.zarr"
    ds = zarr_helpers.open_dataset(store_dir)
    ds_specs = zarr_helpers.get_dataset_specs(shape, store_dir, variable, ds)
    datastores[shape] = ds_specs

In [36]:
df = pd.DataFrame.from_dict(datastores, orient='index')
df

,source,collection_name,variable,shape,lat_resolution,lon_resolution,chunk_size_mb,chunks,dtype,number_coord_chunks,compression
180x360,store_180x360.zarr,180x360,data,"{'time': 1, 'lat': 180, 'lon': 360}",1.005587,1.002786,0.247192,"{'time': 1, 'lat': 90, 'lon': 360}",float64,3,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ..."
569x1138,store_569x1138.zarr,569x1138,data,"{'time': 1, 'lat': 569, 'lon': 1138}",0.316901,0.316623,0.310936,"{'time': 1, 'lat': 143, 'lon': 285}",float64,3,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ..."
1799x3599,store_1799x3599.zarr,1799x3599,data,"{'time': 1, 'lat': 1799, 'lon': 3599}",0.100111,0.100056,0.772476,"{'time': 1, 'lat': 225, 'lon': 450}",float64,3,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ..."
5689x11379,store_5689x11379.zarr,5689x11379,data,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.031646,0.031640,1.933838,"{'time': 1, 'lat': 356, 'lon': 712}",float64,3,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ..."
one_tenth_planet,store_one_tenth_planet.zarr,one_tenth_planet,data,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.010007,0.010005,1.933838,"{'time': 1, 'lat': 356, 'lon': 712}",float64,3,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, ..."


## Test different tiles

Below we test the time to generate tiles from each datastore 10 times.

In [37]:
niters = 1
tile_results_df = {}
dataset_chunk_sizes = {}
results_df = datastores.copy()

for xyz_tile in xyz_tiles:
    results_df[xyz_tile] = {}
    for shape in shapes:
        store_dir = f"store_{shape}.zarr"
        reference, multiscale = False, False

        results_df[xyz_tile][shape] = {
            'time to open (ms)': [],
            'rio reproject (ms)': [],
            'total time (ms)': []
        }
        timings_results = results_df[xyz_tile][shape]
        for iter in range(niters):
            with Timer() as t:
                image_and_timings, cprofile = xarray_tile_reader.tile(
                    store_dir,
                    *xyz_tile,
                    variable=variable,
                )
            total_time = round(t.elapsed * 1000, 2)

            timings = image_and_timings[1]
            timings_results['time to open (ms)'].append(timings['time_to_open']),
            timings_results['rio reproject (ms)'].append(timings['rio.reproject']),
            timings_results['total time (ms)'].append(total_time)
        timings_results['mean time to open (ms)'] = np.mean(timings_results['time to open (ms)'])
        timings_results['mean rio reproject (ms)'] = np.mean(timings_results['rio reproject (ms)']) 
        timings_results['mean total time (ms)'] = np.mean(timings_results['total time (ms)'])

In [39]:
for xyz_tile in xyz_tiles:
    print(f"Results for {xyz_tile}")
    df = pd.DataFrame.from_dict(results_df[xyz_tile], orient='index')
    display(df.sort_values('mean total time (ms)'))
    print()

Results for (0, 0, 0)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
569x1138,[6.54],[32.78],[94.31],6.54,32.78,94.31
180x360,[14.18],[25.32],[124.27],14.18,25.32,124.27
1799x3599,[5.84],[156.12],[216.41],5.84,156.12,216.41
one_tenth_planet,[6.69],[1417.21],[1481.29],6.69,1417.21,1481.29
5689x11379,[6.42],[1517.27],[1579.45],6.42,1517.27,1579.45



Results for (1, 0, 1)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360,[6.31],[13.22],[71.44],6.31,13.22,71.44
569x1138,[5.06],[30.28],[86.29],5.06,30.28,86.29
1799x3599,[6.12],[51.97],[109.77],6.12,51.97,109.77
5689x11379,[6.29],[330.62],[651.83],6.29,330.62,651.83
one_tenth_planet,[7.5],[1152.14],[1222.21],7.50,1152.14,1222.21



Results for (2, 1, 2)


...

,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
1799x3599,[5.11],[12.91],[64.0],5.11,12.91,64.00
5689x11379,[5.12],[13.89],[65.11],5.12,13.89,65.11
one_tenth_planet,[5.24],[14.88],[66.98],5.24,14.88,66.98
569x1138,[6.17],[12.26],[68.12],6.17,12.26,68.12
180x360,[6.97],[15.62],[81.96],6.97,15.62,81.96



Results for (674, 428, 10)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360,[5.06],[12.01],[62.6],5.06,12.01,62.60
1799x3599,[5.12],[13.26],[65.57],5.12,13.26,65.57
one_tenth_planet,[5.27],[15.15],[66.9],5.27,15.15,66.90
5689x11379,[5.73],[14.43],[68.51],5.73,14.43,68.51
569x1138,[10.96],[13.09],[71.31],10.96,13.09,71.31



Results for (1348, 857, 11)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
569x1138,[5.05],[12.45],[63.5],5.05,12.45,63.50
180x360,[5.03],[12.05],[68.82],5.03,12.05,68.82
one_tenth_planet,[6.27],[15.18],[69.51],6.27,15.18,69.51
5689x11379,[7.25],[16.12],[82.46],7.25,16.12,82.46
1799x3599,[5.08],[12.89],[151.89],5.08,12.89,151.89


In [32]:
# Dimensions of the grid
grid_width = 5689
grid_height = 11379

# Earth's circumference at the equator in kilometers
earth_circumference_km = 40075

# Calculate the extent of coverage area in kilometers
extent_width_km = earth_circumference_km / grid_width
extent_height_km = earth_circumference_km / grid_height

print(f"Resolution of the global dataset: {extent_width_km} km x {extent_height_km} km")

Resolution of the global dataset: 7.044296009843558 km x 3.5218384743826348 km
